In [1]:
import json
import os.path

import fsspec
import numpy as np
import xarray as xr
import zarr

In [2]:
fs = fsspec.filesystem('file')
DEFAULT_ZARR_COMPRESSOR = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)

In [3]:
def remove_duplicate_times(dset):
    """Remove duplicate times from dataset."""
    _, index = np.unique(dset['time'], return_index=True)
    return dset.isel(time=index)

In [4]:
def nc_to_zarr(
        ncglob,
        store,
        chunk_dictionary,
        unique_times=True,
        packed=False,
        dtype="int32",
        precision=1e-3,
        fill_value=-32767,
        parallel=False,
        compressor=DEFAULT_ZARR_COMPRESSOR,
        mode="w",
        auto_chunks=True
):
    print("Open dataset from: {}".format(ncglob))
    dataset = xr.open_mfdataset(ncglob, parallel=parallel, decode_times=False)

    if isinstance(dtype, str):
        dtype = {v: dtype for v in dataset.data_vars}

    # Removing duplicate time indexes
    if unique_times:
        dataset = remove_duplicate_times(dataset)

    # Chunking dataset, will define chunking in zarr
    if auto_chunks:
        dataset = dataset.chunk("auto")
    else:
        try:
            dataset = dataset.chunk(chunk_dictionary)
        except ValueError as _:
            f = list(dataset.coords)
            raise CompleteConversionError(
                "Chunks are not valid.\nChunks requested: " + json.dumps(chunk_dictionary) + "\nViable options are: " + str(f))

    # Setting encoding
    encoding = {}
    for data_var in dataset.data_vars:
        encoding[data_var] = {"compressor": compressor, "_FillValue": fill_value}
        if packed:
            encoding[data_var].update(
                {"dtype": dtype.get(data_var, "int32"), "scale_factor": precision}
            )
            # Avoid conflict with encoding read from file
            dataset[data_var].encoding = {}

    # Saving zarr
    print("Creating zarr store: {}".format(store))
    dataset.to_zarr(store=store, mode=mode, encoding=encoding, consolidated=True)

    return dataset

# Version with auto chunks works

In [9]:
nc_to_zarr(
    ncglob='../input/TCCON data from Karlsruhe/ka20100419_20201130.public.nc',
    store='../output/TCCON data from Karlsruhe/test.zarr',
    auto_chunks=True,
    chunk_dictionary={},
    unique_times=True,
    packed=False,
    dtype="float32",
    precision=0.01)

Open dataset from: ../input/TCCON data from Karlsruhe/ka20100419_20201130.public.nc
Creating zarr store: ../output/TCCON data from Karlsruhe/test.zarr


<xarray.Dataset>
Dimensions:           (time: 57377, prior_date: 1223, prior_Height: 71,
                       ak_zenith: 16, ak_P_hPa: 71)
Coordinates:
  * time              (time) float64 1.472e+04 1.472e+04 ... 1.86e+04 1.86e+04
  * prior_date        (prior_date) float64 1.472e+04 1.472e+04 ... 1.86e+04
  * prior_Height      (prior_Height) float64 0.0 1.0 2.0 3.0 ... 68.0 69.0 70.0
  * ak_zenith         (ak_zenith) float64 10.0 15.0 20.0 25.0 ... 75.0 80.0 85.0
  * ak_P_hPa          (ak_P_hPa) float64 1.008e+03 900.9 803.1 ... 0.07 0.06
Data variables: (12/54)
    zobs_km           (time) float32 dask.array<chunksize=(57377,), meta=np.ndarray>
    zmin_km           (time) float32 dask.array<chunksize=(57377,), meta=np.ndarray>
    year              (time) float64 dask.array<chunksize=(57377,), meta=np.ndarray>
    xn2o_ppb_error    (time) float32 dask.array<chunksize=(57377,), meta=np.ndarray>
    xn2o_ppb          (time) float32 dask.array<chunksize=(57377,), meta=np.ndarray>
    xhf_ppt_error     (time) float32 dask.array<chunksize=(57377,), meta=np.ndarray>
    ...                ...
    ak_hf             (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_hdo            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_h2o            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_co2            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_co             (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_ch4            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
Attributes: (12/15)
    More_Information:            https://tccon-wiki.caltech.edu
    TCCON_Reference:             Wunch, D., G. C. Toon, J.-F. L. Blavier, R. ...
    Data_Use_Policy:             https://tccon-wiki.caltech.edu/Network_Polic...
    Auxiliary_Data_Description:  https://tccon-wiki.caltech.edu/Network_Polic...
    Software_Version:            GGG2014
    id:                          ka
    ...                          ...
    Data_DOI:                    10.14291/tccon.ggg2014.karlsruhe01.R1/1182416
    Data_Reference:              Hase, F., T. Blumenstock, S. Dohe, J. Gro�, ...
    Data_Revision:               R1
    creation_date:               2022/01/01
    start_date:                  2010/04/19
    end_date:                    2020/11/30

# Version with time = 10 does not work?

In [10]:
nc_to_zarr(
    ncglob='../input/TCCON data from Karlsruhe/ka20100419_20201130.public.nc',
    store='../output/TCCON data from Karlsruhe/test3.zarr',
    auto_chunks=False,
    chunk_dictionary={'time': 5000},
    unique_times=True,
    packed=False,
    dtype="float32",
    precision=0.01)

Open dataset from: ../input/TCCON data from Karlsruhe/ka20100419_20201130.public.nc
Creating zarr store: ../output/TCCON data from Karlsruhe/test3.zarr


<xarray.Dataset>
Dimensions:           (time: 57377, prior_date: 1223, prior_Height: 71,
                       ak_zenith: 16, ak_P_hPa: 71)
Coordinates:
  * time              (time) float64 1.472e+04 1.472e+04 ... 1.86e+04 1.86e+04
  * prior_date        (prior_date) float64 1.472e+04 1.472e+04 ... 1.86e+04
  * prior_Height      (prior_Height) float64 0.0 1.0 2.0 3.0 ... 68.0 69.0 70.0
  * ak_zenith         (ak_zenith) float64 10.0 15.0 20.0 25.0 ... 75.0 80.0 85.0
  * ak_P_hPa          (ak_P_hPa) float64 1.008e+03 900.9 803.1 ... 0.07 0.06
Data variables: (12/54)
    zobs_km           (time) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    zmin_km           (time) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    year              (time) float64 dask.array<chunksize=(5000,), meta=np.ndarray>
    xn2o_ppb_error    (time) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    xn2o_ppb          (time) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    xhf_ppt_error     (time) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    ...                ...
    ak_hf             (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_hdo            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_h2o            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_co2            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_co             (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
    ak_ch4            (ak_P_hPa, ak_zenith) float64 dask.array<chunksize=(71, 16), meta=np.ndarray>
Attributes: (12/15)
    More_Information:            https://tccon-wiki.caltech.edu
    TCCON_Reference:             Wunch, D., G. C. Toon, J.-F. L. Blavier, R. ...
    Data_Use_Policy:             https://tccon-wiki.caltech.edu/Network_Polic...
    Auxiliary_Data_Description:  https://tccon-wiki.caltech.edu/Network_Polic...
    Software_Version:            GGG2014
    id:                          ka
    ...                          ...
    Data_DOI:                    10.14291/tccon.ggg2014.karlsruhe01.R1/1182416
    Data_Reference:              Hase, F., T. Blumenstock, S. Dohe, J. Gro�, ...
    Data_Revision:               R1
    creation_date:               2022/01/01
    start_date:                  2010/04/19
    end_date:                    2020/11/30